### 1. Detect CUDA version and install the pre-built binaries

In [ ]:
import subprocess
import sys

# Check for GPU
try:
    gpu_info = subprocess.check_output(["nvidia-smi"]).decode("utf-8")
    print("GPU Detected:\n", gpu_info.split('\n')[2])
except:
    raise ValueError("❌ NO GPU DETECTED! Go to Runtime > Change runtime type > T4 GPU")

# Install the library. Try cu121 if cu124 doesn't work
!python -m pip install llama-cpp-python \
  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124 \
  --upgrade --force-reinstall --no-cache-dir

print("Installation Complete.")

✅ GPU Detected:
 | NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 551.3/551.3 MB 311.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 216.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 177.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 178.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 213.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.5
    Uninstalling numpy-2.3.5:
      

## 2. Import libraries

In [ ]:
import gc
import ctypes
import torch
from llama_cpp import Llama

### Free GPU memory method

In [ ]:
def free_memory(model_instance):
    # Close the model connection
    if hasattr(model_instance, 'close'):
        model_instance.close()
        print("Model connection closed.")

    # Delete the Python object to release references
    del model_instance
    print("Python object deleted.")

    # Force Python's Garbage Collector to clear the object
    gc.collect()

    # Force the C++ backend to release memory to the OS
    # This specifically trims the heap memory that llama.cpp allocates
    try:
        libc = ctypes.CDLL("libc.so.6")
        libc.malloc_trim(0)
        print("C++ memory freed")
    except Exception as e:
        print(f"Could not free C++ memory: {e}")

    print("\n GPU Memory should now be freed.")

### 最好用英文题！

# 🚀 Local Inference on GPU (GPU 本地推理)

This notebook runs the **BigJuicyData/Anni-Q4_K_M-GGUF** model locally on your specific GPU instance. (T4 GPU in Google Colab)

</br>

# **🚀 GPU 本地推理**

本 Notebook 將在您特定的 GPU 實例（Google Colab 中的 **T4 GPU**）上本地運行 **BigJuicyData/Anni-Q4_K_M-GGUF** 模型。

</br>

---
</br>

### How to Use (如何使用)
1. **Edit the Prompt:** Scroll down to the code cell and find the variable `PROMPT = "..."`. Replace the text inside with your specific coding question.
2. **Run the Cell:** Press the Play button (▶️) or `Shift + Enter`.
3. **Wait for Stream:** The model will "think" first (showing logic), followed by the code.


### 如何使用
1. **編輯提示詞 (Prompt)：** 向下滾動到程式碼單元格，找到變數 `PROMPT = "..."`。將其中的文字替換為您具體的程式設計問題。
2. **運行單元格：** 按下播放按鈕 (▶️) 或 `Shift + Enter` 鍵。
3. **等待流式輸出：** 模型將先進行「思考」（顯示邏輯），隨後輸出程式碼。

</br>

---

</br>

### Where is the output? (輸出在哪裡?)
The output streams at the **very bottom** of the cell output area. Look for a block formatted like this:

```python
# The output will look like this and is wrapped inside
# ```python
def solution():
    pass
# ```
```

### 輸出在哪裡？
輸出將在單元格輸出區域的最底部以流式傳輸。請尋找如下格式的程式碼塊：

```python
# The output will look like this and is wrapped inside
# ```python
def solution():
    pass
# ```
```

</br>

---

</br>

#### **IF YOU ENCOUNTER AN ERROR, do this!** Go to Runtime > Restart session and run all (如果您遇到錯誤，請這樣做！)
- Restart the session to clear the GPU RAM!

</br>

#### **如果您遇到錯誤，請這樣做！** 前往 Runtime (運行時) > Restart session (重新啟動會話) and run all (並運行全部單元格)

- 重新啟動會話以清除 GPU 記憶體 (RAM)！

</br>
---

Model page (模型頁面):
1. [GGUF](https://huggingface.co/BigJuicyData/coder-final-Q4_K_M-GGUF)
2. **VLLM**: [Huggingface](https://huggingface.co/BigJuicyData/coder-final) , [Modelscope](https://modelscope.cn/models/quanteat/coder-final)
3. [MLX](https://huggingface.co/BigJuicyData/coder-final-mlx-4Bit)

In [ ]:
### IF YOU ENCOUNTER AN ERROR, do this! Go to Runtime > Restart session and run all

REPO_ID = "BigJuicyData/Anni-Q4_K_M-GGUF"
FILENAME = "coder-final-q4_k_m.gguf"
CONTEXT_SIZE = 24000

# --------------------------------------------
# REPLACE THIS WITH YOUR PROGRAMMING QUESTION!
# 用你的编程问题替换此处！最好用英文题。
# --------------------------------------------
PROMPT="""
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.

You may assume that each input would have exactly one solution, and you may not use the same element twice.

You can return the answer in any order.



Example 1:

Input: nums = [2,7,11,15], target = 9
Output: [0,1]
Explanation: Because nums[0] + nums[1] == 9, we return [0, 1].

Example 2:

Input: nums = [3,2,4], target = 6
Output: [1,2]

Example 3:

Input: nums = [3,3], target = 6
Output: [0,1]



Constraints:

    2 <= nums.length <= 104
    -109 <= nums[i] <= 109
    -109 <= target <= 109
    Only one valid answer exists.


Follow-up: Can you come up with an algorithm that is less than O(n2) time complexity?
"""

# Initialize the model
llm = Llama.from_pretrained(
    repo_id=REPO_ID,
    filename=FILENAME,
    n_ctx=CONTEXT_SIZE,
    n_gpu_layers=-1,
    verbose=True
)

# Define the messages (Chat Format)
messages = [
    {
        "role": "system",
        "content": """You are Qwen. You are an expert algorithmic assistant.
1. CORE OBJECTIVE: Solve the user's problem using the most efficient algorithm (optimal Time/Space Complexity).
2. THOUGHT PROCESS: Use your <think> tag to analyze edge cases and plan the logic.
3. OUTPUT: After reasoning, provide the Python solution."""
    },
    {
        "role": "user",
        "content": PROMPT,
    }
]

# Run Inference
print(f"\n Generating with {CONTEXT_SIZE} token limit...\n")

output = llm.create_chat_completion(
    messages=messages,
    max_tokens=CONTEXT_SIZE,
		temperature=0.6,
    top_p=0.95,
    top_k=20,
    min_p=0.0,
    stream=True
)

# Print the streamed response
for chunk in output:
    delta = chunk['choices'][0]['delta']
    if 'content' in delta:
        print(delta['content'], end="", flush=True)

# Info
print("\n可以直接复制以上的代码。代码为 ```python ``` 包容。")

# Free the memory
print("\n-------- Freeing memory --------")
free_memory(llm)
print("-------- Allocated GPU memory should be empty now --------")

llama_model_load_from_file_impl: using device CUDA0 (Tesla T4) - 14980 MiB free
llama_model_loader: loaded meta data with 25 key-value pairs and 443 tensors from /root/.cache/huggingface/hub/models--BigJuicyData--coder-final-Q4_K_M-GGUF/snapshots/ee6cc085a771a1641d29006cb7f721cc1f70c98d/./coder-final-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Coder Final
llama_model_loader: - kv   3:                         general.size_label str              = 15B
llama_model_loader: - kv   4:                          qwen3.block_count u32              = 40
llama_model_loader: - kv   5:                       qwen3.context_leng


 Generating with 24000 token limit...

Okay, I need to solve this two-sum problem. Let's think about the examples given. The task is to find two numbers in the array that add up to the target and return their indices. The constraints say that each input has exactly one solution, and I can't use the same element twice. Oh right, and the follow-up asks for an algorithm better than O(n²), so I need to think about efficient methods.

The brute force approach would be to check every pair of elements, but that's O(n²) time, which is probably too slow for large arrays. The user wants something better. So, what's a more efficient way?

Hmm, using a hash map (dictionary in Python) could work. The idea is to store the values we've seen so far along with their indices. For each number in the array, we check if the complement (target - current number) exists in the hash map. If it does, we return the current index and the index from the hash map. If not, we add the current number to the hash map.

llama_perf_context_print:        load time =     715.25 ms
llama_perf_context_print: prompt eval time =     714.72 ms /   322 tokens (    2.22 ms per token,   450.52 tokens per second)
llama_perf_context_print:        eval time =   58473.03 ms /   942 runs   (   62.07 ms per token,    16.11 tokens per second)
llama_perf_context_print:       total time =   61956.25 ms /  1264 tokens
llama_perf_context_print:    graphs reused =        912



 -------- Freeing memory --------
Model connection closed.
Python object deleted.
C++ memory freed

 GPU Memory should now be freed.
-------- Allocated GPU memory should be empty now --------
